# Imersão Dados - Aula 1: Introdução

<p>Olá!!! Bem-vindos ao meu notebook da Imersão dados da Alura. Essa iniciative é fantástica para trazer a ciência de dados a profissionais de outras àreas</p>
<p>O intuito desse notebook é "traduzir" os códigos, transformações de dados e visualizações do <code>python</code> para <code>R</code></p>

## O Problema
<p>O problema que vamos investigar é a precificação de imóveis a partir de dados provenientes de uma base de dados. Teremos várias características dos imóveis (área, número de cômodos, localização), essas serão nossas <strong>features</strong>, ou variáveis que serão usadas para fazermos a previsão. O preço dos imóveis será o nosso <strong>target</strong>, ou seja, a resposta que desejamos que o modelo seja capaz de estimar. Como a resposta será contínua (isto é, o preço pode variar de 0 reais a Infinito), trata-se de um problema de <strong>regressão</strong>.</p>

<p>Para começar a manipular os dados e resolver o problema, precisamos, claro, importá-los</p>

In [1]:
library(tidyverse) #carrega a biblioteca do tidyverse, a biblia da manipulacao e visualizacao de dados em R

dados <- read_csv("https://gist.githubusercontent.com/tgcsantos/3bdb29eba6ce391e90df2b72205ba891/raw/22fa920e80c9fa209a9fccc8b52d74cc95d1599b/dados_imoveis.csv")
glimpse(dados) #visualizar a estrutura dos dados

<p>Temos 8 variáveis diferentes nesse dataset:</p>
<table>
<tr>
    <th>Variável</th>
    <th>Tipo Atual</th>
    <th>Tipo Correto</th>
</tr>
<tr>
    <td>Rua</td>
    <td>Caracter</td>
    <td>Caracter</td>
</tr>
<tr>
    <td>Bairro</td>
    <td>Caracter</td>
    <td>Caracter</td>
</tr>
<tr>
    <td>Cidade</td>
    <td>Caracter</td>
    <td>Caracter</td>
</tr>
<tr>
    <td>Metragem</td>
    <td>double</td>
    <td>double</td>
</tr>
<tr>
    <td>Quartos</td>
    <td>double</td>
    <td>int</td>
</tr>
<tr>
    <td>Banheiros</td>
    <td>double</td>
    <td>int</td>
</tr>
<tr>
    <td>Vagas</td>
    <td>double</td>
    <td>int</td>
</tr>
<tr>
    <td>Valor</td>
    <td>Caracter</td>
    <td>double</td>
</tr>
</table>

Logo de cara já aparecem alguns problemas:
1. O número de quartos, vagas e banheiros deve ser sempre um número inteiro (e não negativo)
2. O Valor está formatado como texto devido ao "R$" que o precede

Mais a frente podemos cogitar modelar a cidade como um fator para determinar sua influência no preço final do imóvel, mas para isso é necessário que haja um número relevante de casos por cidade.

In [19]:
#Consertando as classes das variáveis
dados$Quartos <- as.integer(dados$Quartos)
dados$Banheiros <- as.integer(dados$Banheiros)
dados$Vagas <- as.integer(dados$Vagas)
#Carregar biblioteca para manipulacao de strings
library(stringr)
dados$Valor <- str_remove_all(dados$Valor, "R\\$ ") %>% #necessario escapar o $ por ser um caracter especial
                str_remove_all("\\.") %>%
                as.numeric()

#Verificando a estrutura novamente
glimpse(dados)

## Desafios

><strong>Desafios desta aula</strong><br>
1 - Realizar a média da metragem para cada um dos bairros. (Proposto pelo Paulo)<br>
2 - Duas formas de selecionar os dados por bairro (consultar os métodos na documentação do Pandas) (Proposto pelo Thiago)<br>
3 - Explorar alguns gráficos na documentação e aplicar nas demais colunas do DF, assim como tentar colocar alguma conclusão. (Proposto pelo Thiago)<br>
4 - Pegar outras estatísticas dos dados (como média, mediana, mim, max). (Proposto pela Vivian)<br>
5 - Descobrir quais são os bairros que não tem nome de rua. (Proposto pela Vivian)<br>

In [52]:
#Desafio 1: Media
dados %>% 
    group_by(Bairro) %>%
    summarize(Media_m2 = mean(Metragem))

In [41]:
#Desafio 2: Selecionar os dados por bairro
filter(dados, Bairro == "Vila Mariana")

#ou

subset(dados, Bairro == "Vila Mariana")

In [22]:
#Desafio 3: Graficos Exploratorios

#Relacao entre metragem e valor
dados %>%
    ggplot(aes(Metragem, Valor))+
    geom_point()+ #desenha os pontos
    geom_smooth() -> p #adiciona uma linha de tendencia e salva em uma variavel p
p

<p>Percebemos um problema clássico, a variação no eixo y é muito maior do que no eixo x. Para uma visualização mais adequada, podemos usar uma transformação logaritmica da escala do eixo y</p>

In [23]:
p+scale_y_log10()

<p>Ainda impossível de entender, precisamos filtrar os valor extremos</p>

In [51]:
options(scipen = 999)
dados %>%
        filter(Valor >= 10^5, Metragem < 10^4 ) %>%
        ggplot(aes(Metragem, Valor))+
        geom_point()+
        geom_smooth()+
        scale_y_log10()+
        ggtitle("Relação entre área e valor de imóveis de até 10.000 m^2")

<p>Agora podemos perceber que a maioria dos imóveis têm entre 0 e 1000 m^2, e nesse intervalo temos uma correlação entre preço e área.</p>

In [54]:
#4 - Pegar outras estatísticas dos dados (como média, mediana, mim, max)
summary(dados[4:8])

In [33]:
#5 - Descobrir quais são os bairros que não tem nome de rua
dados %>%
    filter(is.na(Rua))%>%
    group_by(Bairro)%>%
    summarize(n_de_ruas_faltantes = n())%>%
    arrange(desc(n_de_ruas_faltantes))